# core

> Core Client

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import show_doc

In [ ]:
#| export
import requests
import json
import os
from google.cloud import vision
from google.oauth2 import service_account
from PIL import Image
import io
from google.cloud.vision import AnnotateImageResponse
import shutil
from bs4 import BeautifulSoup
from xml.dom.minidom import parseString
from dotenv import load_dotenv

In [ ]:
#| export
class IIIFClient:

    ID_PREFIX = "http://example.org/iiif/abc"
    title = "GOCR"

    def __init__(self, cred_file_path: str, id_prefix: str = None) -> None:
        self.cred_file_path = cred_file_path

        if id_prefix is not None:
            self.ID_PREFIX = id_prefix

    def create_manifest3_by_gocr(self, url: str, output_iiif_manifest_file_path: str, tmp_dir: str = "./tmp", debug=False, title = None) -> None:
        """
        URLから画像をダウンロードし、GOCR APIを使用してManifest3ファイルを作成する。

        Parameters
        ----------
        url : str
            画像のURL
        output_iiif_manifest_file_path : str
            出力するIIIFマニフェストファイルのパス
        tmp_dir : str
            一時ディレクトリのパス
        debug : bool
            デバッグモードの有無
        title : str
            マニフェストのタイトル

        Returns
        -------
        None
        """
        tmp_img_path = self._download_image(url, tmp_dir)

        if tmp_img_path is not None:
            self.create_manifest3_local_by_gocr(tmp_img_path, output_iiif_manifest_file_path, debug=debug, title=title, url=url)

    def _download_image(self, url: str, tmp_dir: str) -> str:
        """
        画像をURLからダウンロードし、一時ディレクトリに保存する。
        """
        tmp_img_path = os.path.join(tmp_dir, "01/images", url.replace("/", "%2F"))
        if not os.path.exists(tmp_img_path):
            os.makedirs(os.path.dirname(tmp_img_path), exist_ok=True)
            try:
                headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
                response = requests.get(url, headers=headers)
                response.raise_for_status()  # ステータスコードが200番台でない場合にエラーを発生させる
                with open(tmp_img_path, 'wb') as f:
                    f.write(response.content)

                # ダウンロードしたファイルが画像として認識されるか確認する
                Image.open(tmp_img_path).verify()
            except requests.RequestException as e:
                print(f"リクエスト中にエラーが発生しました: {e}")
                return None
            except IOError:
                print(f"ダウンロードされたファイルが画像として認識できません: {tmp_img_path}")
                return None
        return tmp_img_path
    
    def create_manifest3_local_by_gocr(self, input_image_file_path: str, output_iiif_manifest_file_path: str, debug=False, title: str = None, url = None) -> None:
        """
        ローカルに保存された画像からGOCR APIを使用してManifest3ファイルを作成する。
        """

        if title is not None:
            self.title = title

        if url is None:
            self.url = input_image_file_path
        else:
            self.url = url

        self.input_image_file_path = input_image_file_path

        self.output_iiif_manifest_file_path = output_iiif_manifest_file_path
     
        self.get_image_dimensions()
        self.perform_ocr()

        if debug:
            print(json.dumps(self.ocr_result_json, indent=4, ensure_ascii=False))
        self.create_manifest3_data()
        self.write_manifest3_file()
        self.copy_image_to_output()

    def get_image_dimensions(self) -> tuple:
        """
        画像の寸法を取得する。
        """

        image_path = self.input_image_file_path

        with Image.open(image_path) as im:
            self.img_width, self.img_height = im.size
        
    def perform_ocr(self) -> dict:
        """
        Google Cloud Vision APIを使用してOCRを実行し、結果をJSONとして返す。
        """
        image_path = self.input_image_file_path
        credentials = service_account.Credentials.from_service_account_file(self.cred_file_path)
        client = vision.ImageAnnotatorClient(credentials=credentials)
        with io.open(image_path, 'rb') as image_file:
            content = image_file.read()
        image = vision.Image(content=content)
        response = client.text_detection(image=image)
        json_str = AnnotateImageResponse.to_json(response)
        self.ocr_result_json = json.loads(json_str)
    
    def create_manifest3_data(self) -> dict:
        """
        OCR結果からManifest3データを作成する。
        """

        json_data = self.ocr_result_json

        img_width = self.img_width
        img_height = self.img_height

        annotations = []

        annotations_ = json_data["textAnnotations"][1:]

        id_prefix = self.ID_PREFIX
        
        canvas_id = f"{id_prefix}/canvas/p1"

        for annotation_ in annotations_:

            min_x = float('inf')
            min_y = float('inf')
            max_x = float('-inf')
            max_y = float('-inf')

            # 全ての頂点をループして最小値と最大値を見つける
            for vertex in annotation_["boundingPoly"]["vertices"]:
                x = vertex["x"]
                y = vertex["y"]
                min_x = min(min_x, x)
                min_y = min(min_y, y)
                max_x = max(max_x, x)
                max_y = max(max_y, y)

            # 幅と高さを計算
            width = max_x - min_x
            height = max_y - min_y

            xywh = (min_x, min_y, width, height)

            xywh = ','.join(map(str, xywh))

            annotation = {
                "body": {
                    "type": "TextualBody",
                    "value": f'[{str(len(annotations) + 1).zfill(5)}] {annotation_["description"]}',
                    "format": "text/plain"
                },
                "id": f"{canvas_id}#xywh={xywh.replace(',', '/')}",
                "type": "Annotation",
                "motivation": "commenting",
                "target": f"{canvas_id}#xywh={xywh}"
            }
            annotations.append(annotation)

        

        manifest3 = {
            "@context": "http://iiif.io/api/presentation/3/context.json",
            "id": f"{id_prefix}/manifest",
            "label": {
                "none": [
                    self.title
                ]
            },
            "type": "Manifest",
            "items": [
                {
                    "id": canvas_id,
                    "type": "Canvas",
                    "label": {
                        "none": [
                            "[1]"
                        ]
                    },
                    "height": img_height,
                    "width": img_width,
                    "items": [
                        {
                        "id": f"{id_prefix}/annotation/p0001-image",
                        "type": "AnnotationPage",
                        "items": [
                            {
                            "body": {
                                "id": self.url,
                                "type": "Image",
                                "format": "image/jpeg",
                                "height": img_height,
                                "width": img_width,
                            },
                            "id": f"{id_prefix}/annotation/p0001-image/anno",
                            "type": "Annotation",
                            "motivation": "painting",
                            "target": canvas_id
                            }
                        ]
                        }
                    ],
                    "annotations": [
                        {
                            "id": f"{canvas_id}/curation",
                            "type": "AnnotationPage",
                            "items": annotations
                        }
                    ]
                }
            ]
        }

        self.manifest_data_json = manifest3
    
    def write_manifest3_file(self) -> None:
        """
        Manifest3データをファイルに書き込む。
        """

        manifest3 = self.manifest_data_json
        output_path = self.output_iiif_manifest_file_path

        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        with open(output_path, 'wt') as f:
            json.dump(manifest3, f, indent=4, ensure_ascii=False)

    def copy_image_to_output(self) -> None:
        """
        出力ディレクトリに画像をコピーする。
        """

        image_path = self.input_image_file_path
        output_path = self.output_iiif_manifest_file_path

        output_image_path = os.path.join(os.path.dirname(output_path), "images", "demo.jpg")

        if image_path == output_image_path:
            return

        os.makedirs(os.path.dirname(output_image_path), exist_ok=True)

        shutil.copyfile(image_path, output_image_path)
        

In [ ]:
#| export
class TEIClient:
    def convert_manifest3_annotations_to_zones(self, input_iiif_manifest_file_path: str, output_tei_xml_file_path: str) -> None:
        """
        ManifestファイルをTEIのXML形式に変換するメソッド

        Args:
            input_iiif_manifest_file_path (str): 入力となるManifestファイルのパス
            output_tei_xml_file_path (str): 出力となるTEIのXMLファイルのパス

        Returns:
            None
        """

        manifest = self._load_json(input_iiif_manifest_file_path)

        annotations = manifest['items'][0]["annotations"][0]["items"]

        # XMLテンプレートの作成
        xml_string = self._create_initial_xml(manifest)

        # BeautifulSoupでXML操作
        soup = BeautifulSoup(xml_string, 'xml')
        self._append_zones_to_xml(soup, annotations)

        # XMLを整形してファイルに書き出し
        self._write_formatted_xml(soup, output_tei_xml_file_path)

    def _load_json(self, path: str) -> dict:
        """
        JSONファイルを読み込むメソッド

        Args:
            path (str): JSONファイルのパス

        Returns:
            dict: JSONファイルの内容
        """

        with open(path, 'r') as f:
            return json.load(f)
        
    def _create_initial_xml(self, manifest: dict) -> str:
        """
        初期のXMLテンプレートを作成するメソッド

        Args:
            manifest (dict): Manifestファイルの内容
        
        Returns:
            str: 初期のXMLテンプレート
        """

        title = manifest["label"]["none"][0] if "none" in manifest["label"] else "Example"

        graphic_url = manifest['items'][0]["items"][0]["items"][0]['body']['id']
        return f'''<?xml version="1.0" encoding="UTF-8"?>
<?xml-model href="https://tei-c.org/release/xml/tei/custom/schema/relaxng/tei_all.rng" type="application/xml" schematypens="http://relaxng.org/ns/structure/1.0"?>
<TEI xmlns="http://www.tei-c.org/ns/1.0">
    <teiHeader>
        <fileDesc>
            <titleStmt>
                <title>{title}</title>
            </titleStmt>
            <publicationStmt>
                <p>Example</p>
            </publicationStmt>
            <sourceDesc>
                <p>Example</p>
            </sourceDesc>
        </fileDesc>
        <profileDesc>
            <creation>
                <listChange>
                    <change></change>
                </listChange>
            </creation>
        </profileDesc>
    </teiHeader>
    <facsimile>
        <surface>
            <graphic url="{graphic_url}"></graphic>
        </surface>
    </facsimile>    
</TEI>'''
    
    def _append_zones_to_xml(self, soup: BeautifulSoup, annotations: list) -> None:
        """
        BeautifulSoupを使用してzoneタグを追加するメソッド

        Args:
            soup (BeautifulSoup): BeautifulSoupオブジェクト
            annotations (list): Manifestファイルのannotationsのリスト
        
        Returns:
            None
        """

        surface = soup.find('surface')

        for i, annotation in enumerate(annotations):
            zone = soup.new_tag('zone')
            zone['xml:id'] = f"a_{str(i).zfill(4)}"
            xywh = annotation['target'].split("#xywh=")[1]
            coords = xywh.split(',')

            # 座標情報の設定
            zone['ulx'], zone['uly'] = coords[0], coords[1]
            zone['lrx'] = str(int(coords[0]) + int(coords[2]))
            zone['lry'] = str(int(coords[1]) + int(coords[3]))

            # テキスト内容の設定
            text = annotation['body']['value']
            text = text.split("]")[1].strip() if "]" in text else text
            seg = soup.new_tag('seg')
            seg.string = text
            zone.append(seg)

            surface.append(zone)

    # 整形したXMLをファイルに書き出す
    def _write_formatted_xml(self, soup: BeautifulSoup, opath: str) -> None:
        """
        整形したXMLをファイルに書き出す

        Args:
            soup (BeautifulSoup): BeautifulSoupオブジェクト
            opath (str): 出力ファイルのパス

        Returns:
            None
        """

        formatted_str = self._format_xml_minidom(str(soup))
        os.makedirs(os.path.dirname(opath), exist_ok=True)
        with open(opath, 'w') as f:
            f.write(formatted_str)

    # minidomを使用してXMLを整形する
    def _format_xml_minidom(self, xml_string: str) -> str:
        """
        minidomを使用してXMLを整形する

        """

        dom = parseString(xml_string)
        pretty_xml_as_string = dom.toprettyxml()

        # 空行を除去
        return '\n'.join(line for line in pretty_xml_as_string.split('\n') if line.strip())

In [ ]:
#| export
class CoreClient:

    @staticmethod
    def load_env() -> None:
        """
        環境変数を読み込む。
        """
        load_dotenv()
        return os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

    @staticmethod
    def create_tei_xml_with_gocr(url: str, output_tei_xml_file_path: str, cred_file_path: str, tmp_dir: str = "./tmp", debug=False, title: str = None) -> None:
        """
        URLから画像をダウンロードし、GOCR APIを使用してTEIのXMLファイルを作成する。

        Parameters
        ----------
        url : str
            画像のURL
        output_tei_xml_file_path : str
            出力するTEIのXMLファイルのパス
        cred_file_path : str
            GOCR APIの認証情報ファイルのパス
        tmp_dir : str
            一時ディレクトリのパス
        debug : bool
            デバッグモードの有無
        title : str
            マニフェストのタイトル

        Returns
        -------
        None
        """

        iiif_client = IIIFClient(cred_file_path)

        input_manifest_file_path = output_tei_xml_file_path.replace(".xml", ".json")

        iiif_client.create_manifest3_by_gocr(url, input_manifest_file_path, tmp_dir, debug, title)

        tei_client = TEIClient()
        tei_client.convert_manifest3_annotations_to_zones(input_manifest_file_path, output_tei_xml_file_path)

In [ ]:
show_doc(CoreClient.load_env)

---

### CoreClient.load_env

>      CoreClient.load_env ()

*環境変数を読み込む。*

In [ ]:
show_doc(IIIFClient.create_manifest3_by_gocr)

---

[source](https://github.com/nakamura196/iiif_tei_py/blob/main/iiif_tei_py/iiif.py#L31){target="_blank" style="float:right; font-size:smaller"}

### IIIFClient.create_manifest3_by_gocr

>      IIIFClient.create_manifest3_by_gocr (url:str,
>                                           output_iiif_manifest_file_path:str,
>                                           tmp_dir:str='./tmp', debug=False,
>                                           title=None)

*URLから画像をダウンロードし、GOCR APIを使用してManifest3ファイルを作成する。*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| url | str |  | 画像のURL |
| output_iiif_manifest_file_path | str |  | 出力するIIIFマニフェストファイルのパス |
| tmp_dir | str | ./tmp | 一時ディレクトリのパス |
| debug | bool | False | デバッグモードの有無 |
| title | NoneType | None | マニフェストのタイトル |
| **Returns** | **None** |  |  |

In [ ]:
show_doc(TEIClient.convert_manifest3_annotations_to_zones)

---

[source](https://github.com/nakamura196/iiif_tei_py/blob/main/iiif_tei_py/tei.py#L14){target="_blank" style="float:right; font-size:smaller"}

### TEIClient.convert_manifest3_annotations_to_zones

>      TEIClient.convert_manifest3_annotations_to_zones
>                                                        (input_iiif_manifest_fi
>                                                        le_path:str, output_tei
>                                                        _xml_file_path:str)

*ManifestファイルをTEIのXML形式に変換するメソッド

Args:
    input_iiif_manifest_file_path (str): 入力となるManifestファイルのパス
    output_tei_xml_file_path (str): 出力となるTEIのXMLファイルのパス

Returns:
    None*

In [ ]:
show_doc(CoreClient.create_tei_xml_with_gocr)

---

### CoreClient.create_tei_xml_with_gocr

>      CoreClient.create_tei_xml_with_gocr (url:str,
>                                           output_tei_xml_file_path:str,
>                                           cred_file_path:str,
>                                           tmp_dir:str='./tmp', debug=False,
>                                           title:str=None)

*URLから画像をダウンロードし、GOCR APIを使用してTEIのXMLファイルを作成する。*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| url | str |  | 画像のURL |
| output_tei_xml_file_path | str |  | 出力するTEIのXMLファイルのパス |
| cred_file_path | str |  | GOCR APIの認証情報ファイルのパス |
| tmp_dir | str | ./tmp | 一時ディレクトリのパス |
| debug | bool | False | デバッグモードの有無 |
| title | str | None | マニフェストのタイトル |
| **Returns** | **None** |  |  |

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()